In [1]:
import pandas as pd
import numpy as np
from skimage.measure import label, regionprops

Method cleaningTable merupakan method utama yang berfungsi untuk membersihkan table excel

Nilai kembalian dari method ini adalah array dua dimensi. Dimensi menunjukkan sheet pada file excel, dimensi yang kedua menunjukkan tabel yang ada pada sheet tersebut

In [21]:
def cleaningTable(data_path):

    xl = pd.ExcelFile(data_path)
    res = len(xl.sheet_names)

    list_sheet = []

    #=========read excel file
    for i in xl.sheet_names:
          df = pd.read_excel(data_path, sheet_name=i, engine='openpyxl')
          list_sheet.append(df)

    #=========remove sheet kosong
    list_clean_sheet = []
    for i in list_sheet:
      size = i.shape
      if ((size[0] == 0) and (size[1] == 0)):
        continue
      else:
        list_clean_sheet.append(i)

    #========parsing table
    list_data = []
    for i in list_clean_sheet:
      list_data.append(parseTabel(i))

    #========remove None Row in table
    for i in list_data:  
      for index,y in enumerate(i):
        i[index] = removeNoneRow(y)
  

    #========delete those that are not tables
    list_clean_parsing = []
    for i in list_data:
      temporary = []
      for y in i :
        check = isItTable(y,1)
        if (check):
          temporary.append(y)
        else:
          continue
      list_clean_parsing.append(temporary)

    list_data_nonDuplicate = []
    for i in list_data:
      temporary = []
      counter2 = 0
      for y in range(len(i)):
        counter = 0
        list_answer = []
        
        while (counter <= (len(i)-1)):
          if (y != counter):
            if (similarityTable(i[y], i[counter]) == 1):
              if (checkSimilarityDataTable(i[y], i[counter])):
                list_answer.append(True)
              else:
                list_answer.append(False)
            elif (similarityTable(i[y], i[counter]) == 2):
              if (counter2 == 0):
                list_answer.append(False)
              else:
                list_answer.append(True)
              counter2 += 1
            else:
              list_answer.append(False)
          counter += 1

        if (True in list_answer):
          continue
        else:
          temporary.append(i[y])
      list_data_nonDuplicate.append(temporary)  
      
  
    for i in list_data_nonDuplicate:
      for index,y in enumerate(i):
        i[index] = removeTitle(y)
    
    list_table = []
    for i in list_data_nonDuplicate:
      temporary = []
      for y in i:
        table = np.array(y.T.reset_index().values.T.tolist())
        if (isItTable(table,1)):
          temporary.append(y)
      list_table.append(temporary)

    for i in list_table:
      for index,y in enumerate(i):
        i[index] = checkMergedCell(y)

    list_table_result = []
    for i in list_table:
      temporary = []
      for y in i:
        table = np.array(y.T.reset_index().values.T.tolist())
        if (isItTable(table,1)):
          temporary.append(y)
      list_table_result.append(temporary)
    #return list_clean_parsing

    #return list_data    
    
    #return list_data_nonDuplicate

    return list_table_result

    #return list_clean_parsing

method cleaning by sheetname

In [39]:
def cleaningTablebySheetname(data_path, sheet_Name):

    #xl = pd.ExcelFile(data_path)
    #res = len(xl.sheet_names)

    #list_sheet = []

    #=========read excel file
    df = pd.read_excel(data_path, sheet_name=sheet_Name, engine='openpyxl')

    #========parsing table
    list_data = parseTabel(df)

    #========remove None Row in table
    for index,i in enumerate(list_data):
      list_data[index] = removeNoneRow(i)
    
    #========delete those that are not tables
    list_clean_parsing = []
    for i in list_data:
      check = isItTable(i,1)
      if (check):
        list_clean_parsing.append(i)
      else:
        continue
    
    list_data_nonDuplicate = [] 
    temporary = []
    counter2 = 0
    for y in range(len(list_data)):
      counter = 0
      list_answer = []
        
      while (counter <= (len(list_data)-1)):
        if (y != counter):
          if (similarityTable(list_data[y], list_data[counter]) == 1):
            if (checkSimilarityDataTable(list_data[y], list_data[counter])):
              list_answer.append(True)
            else:
              list_answer.append(False)
          elif (similarityTable(list_data[y], list_data[counter]) == 2):
            if (counter2 == 0):
              list_answer.append(False)
            else:
              list_answer.append(True)
            counter2 += 1
          else:
            list_answer.append(False)
        counter += 1

      if (True in list_answer):
        continue
      else:
        list_data_nonDuplicate.append(list_data[y])
    
    for index,y in enumerate(list_data_nonDuplicate):
      list_data_nonDuplicate[index] = removeTitle(y)
    
    list_table = []
    for i in list_data_nonDuplicate:
      table = np.array(i.T.reset_index().values.T.tolist())
      if (isItTable(table,1)):
        list_table.append(i)

    for index,y in enumerate(list_table):
      list_table[index] = checkMergedCell(y)

    list_table_result = []
    for i in list_table:
      table = np.array(y.T.reset_index().values.T.tolist())
      if (isItTable(table,1)):
        list_table_result.append(i)

    #return list_clean_parsing

    #return list_data    
    
    #return list_data_nonDuplicate

    return list_table_result

    #return list_table

Method parseTabel merupakan method untuk memprediksi tabel apa saja yang terdapat pada file excel tersebut. Library yang digunakan untuk memprediksi adalah skimage.measure

nilai kembaliannya berupa list yang berisi tabel-tabel setelah diparsing dalam bentuk data frame

In [3]:
def parseTabel(data):
    data_array = np.array(data.T.reset_index().values.T.tolist())
    #binary_rep = np.array(data_array.notnull().astype('int'))
    data_array_copy = data_array.copy()
    size = data_array.shape

    for i in range(size[0]):
      for y in range(size[1]):
        if (str(data_array[i,y]) == 'nan' or 'Unnamed' in str(data_array[i,y])):
          data_array[i,y] = 0
        else:
          data_array[i,y] = 1


    list_of_dataframes = []
    l = label(data_array)

    region = regionprops(l)
    
    #for i in region:
    #  print(i.bbox)

    for i in range(len(region)):
      firstRow = region[i].bbox[0]
      firstColumn = region[i].bbox[1]  

      if (i < (len(region) - 1)):
        secondRow = region[i+1].bbox[0]
        secondColumn = region[i+1].bbox[1]

        if (firstRow == secondRow):
          if (firstColumn == secondColumn):
            slice_data = data_array_copy[region[i].bbox[0]:region[i+1].bbox[2],region[i].bbox[1]:region[i+1].bbox[3]]
            list_of_dataframes.append(pd.DataFrame(slice_data[1:], columns = slice_data[0]))
          else:
            if (i < (len(region) - 2)):
              thirdRow = region[i+2].bbox[0]
              thirdColumn = region[i+2].bbox[1]
              if (thirdColumn == secondColumn and thirdRow == firstRow):
                slice_data = data_array_copy[region[i].bbox[0]:region[i+2].bbox[2],region[i].bbox[1]:region[i+2].bbox[3]]
                list_of_dataframes.append(pd.DataFrame(slice_data[1:], columns = slice_data[0]))
              else:
                slice_data = data_array_copy[region[i].bbox[0]:region[i+1].bbox[2],region[i].bbox[1]:region[i+1].bbox[3]]
                list_of_dataframes.append(pd.DataFrame(slice_data[1:], columns = slice_data[0]))
            else:
              slice_data = data_array_copy[region[i].bbox[0]:region[i+1].bbox[2],region[i].bbox[1]:region[i+1].bbox[3]]
              list_of_dataframes.append(pd.DataFrame(slice_data[1:], columns = slice_data[0]))
        else:
          slice_data = data_array_copy[region[i].bbox[0]:region[i].bbox[2],region[i].bbox[1]:region[i].bbox[3]]
          list_of_dataframes.append(pd.DataFrame(slice_data[1:], columns = slice_data[0]))
      else:
        slice_data = data_array_copy[region[i].bbox[0]:region[i].bbox[2],region[i].bbox[1]:region[i].bbox[3]]
        list_of_dataframes.append(pd.DataFrame(slice_data[1:], columns = slice_data[0]))

      #counter = i+1
      #while (counter < len(region)):
      #  startColumn_test = region[counter].bbox[1]
      #  lastRow_test = region[counter].bbox[2]
        
      #  if ((startColumn1 == startColumn_test) and (lastRow1 == lastRow_tes)):
      #    continue 
      #  else


    #for index,s in enumerate(regionprops(l)):
        
        #slice_data = data_array_copy[s.bbox[0]:s.bbox[2],s.bbox[1]:s.bbox[3]]
        #list_of_dataframes.append(pd.DataFrame(slice_data[1:], columns = slice_data[0]))
        #list_of_dataframes.append(data.iloc[s.bbox[0]:s.bbox[2],s.bbox[1]:s.bbox[3]])

    list_parseTable = []

    for i in list_of_dataframes:
        if (isItTable(i,1)):
            list_parseTable.append(i)
        else:
            continue

    list_cleanParsing = []
    for i in list_parseTable:
      if (removeNoneTable(i)):
        continue
      else:
        list_cleanParsing.append(i)

    return list_cleanParsing
    #return list_parseTable


Method isItTable merupakan method yang berfungsi untuk mengecek apakah tiap tabel hasil parsing pada method parsTabel merupakan benar-benar tabel atau bukan.

Logika yang digunakan untuk method ini adalah jika tabel tersebut memliki baris kurang dari atau sama dengan 1 maka otomatis bukan merupakan tabel. Lalu jika jika kondisi tersebut ternyata salah, maka kemudian akan dicek kembali. Jika baris dan kolom tabel tersebut kurang dari 2, maka otomatis bukan merupakan tabel

Nilai kembalian dari method ini adalah boolean, artinya jika setelah dicek ternyata bukan tabel, maka nilai kembaliannya adalah "False". Kemudian sebaliknya, jika setelah dicek ternyata merupakan tabel, maka nilai kembaliannya adalah "True"

In [4]:
def isItTable(data, maxRow):
    row,column = data.shape

    #if (row <= 1):
    #    return False
    #elif ():
      
    #else:
    #    if (row <= 2 and column <= 2):
    #        return False
    #    else:
    #        return True

    if (row < maxRow):
      return False
    elif (row <= 2 and column <= 2):
      return False
    else:
      return True

In [5]:
def getIndexColumn(data):
  list_index_column = []
  for i in data:
    try:
       a = int(i)
       list_index_column.append(a)
    except:
      continue
  result = 0
  
  dif_value = []
  for i in range(len(list_index_column)-1):
    dif_value.append(list_index_column[i]-list_index_column[i+1])

  if (len(list_index_column) <= 1):
    result = False
    #print('False 1')
  elif (checkZeroDif(dif_value)):
    result = True
  else:
    different_value = list_index_column[1] - list_index_column[0]
    
    counter = 0
    for i in range(len(list_index_column)-1):
      #print(i)
      check = list_index_column[i+1] - list_index_column[i]
      if (check != different_value):
        result = False
        #print('False 2')
        break
      else:
        if ((i+1) == len(list_index_column)-1):
          if (list_index_column[0] >= 1000):
            result = False
          else:
            result = True
          #print('True')
  return result

def checkZeroDif(data):
  list_boolean = []
  for i in data:
    if (i == 0):
      list_boolean.append(True)
    else:
      list_boolean.append(False)

  if (False in list_boolean):
    return False
  else:
    return True

def getNoneRow(data):
  list_noneData = []
  counter = 0
  
  for i in data:
    if (str(i) == 'nan' or 'Unnamed' in str(i)):
      list_noneData.append(counter)
    counter += 1
  
  if (len(list_noneData) ==  len(data)):
    return True
  else:
    return False

def removeNoneRow(data):
  data_array = np.array(data.T.reset_index().values.T.tolist())
  data_result = []

  for i in range(len(data_array)):
    if (getNoneRow(data_array[i,:]) or getIndexColumn(data_array[i,:])):
      #print('halo')
      continue
    else:
      data_result.append(data_array[i,:])

  result = np.array(data_result)

  return pd.DataFrame(result[1:], columns = result[0])

In [6]:
def removeNoneTable(table):
  data_array = np.array(table.T.reset_index().values.T.tolist())

  row = data_array.shape[0]
  column = data_array.shape[1]

  counter = 0
  for i in range(row):
    for y in range(column):
      if (data_array[i,y] == None or data_array[i,y] == 'NaN' or data_array[i,y] == 'nan' or data_array[i,y] == 'None' or 'Unnamed' in data_array[i,y]):
        counter += 1
  
  if (counter == (row*column)):
    return True
  else:
    return False

Method checkSimilarityDataTable dan similarityTable merupakan method untuk membandingkan antara dua tabel apakah sama atau tidak. Hal ini dilakukan untuk mengatasi masalah ketika menjalankan method parseTabel, karena bisa saja ketika melakukan parsing, sebuah tabel bisa saja dipisah menjadi 2 tabel

In [7]:
def checkSimilarityDataTable(table1, table2):
  np_array1 = np.array(table1.T.reset_index().values.T.tolist())
  np_array2 = np.array(table2.T.reset_index().values.T.tolist())

  rowSize1 = np_array1.shape[0]
  columnSize1 = np_array1.shape[1]
  rowSize2 = np_array2.shape[0]
  columnSize2 = np_array2.shape[1]
  
  number_true_value = 0
  counter = 0
  for i in range(rowSize1):
    for j in range(columnSize1):
      loc = np.where(np_array2 == np_array1[i,j])
      if (len(loc[0]) != 0):
        #print('True')
        counter += 1

  result = counter / (rowSize1*columnSize1)
  if (result == 1):
    return True
    #print('True')
  else: 
    return False
    #print('False')

def similarityTable(table1, table2):
  #print("HALO HALO HALO")
  np_array1 = np.array(table1.T.reset_index().values.T.tolist())
  np_array2 = np.array(table2.T.reset_index().values.T.tolist())

  check1 = 0
  check2 = 0

  rowSize1 = np_array1.shape[0]
  columnSize1 = np_array1.shape[1]
  rowSize2 = np_array2.shape[0]
  columnSize2 = np_array2.shape[1]

  #===== check table1 in table2
  if (np_array2.shape[0] >= np_array1.shape[0]):
    if (np_array1.shape[1] > np_array2.shape[1]):
      check1 = 0
      #print("hai")
    elif ((np_array1.shape[1] == np_array2.shape[1]) and (np_array1.shape[0] == np_array2.shape[0])):
      if  (checkSimilarityDataTable(table1, table2)):
        check1 = 2
      else:
        check1 = 0
    else:
      #print("test")
      startValue = np_array1[0,0]
      loc = np.where(np_array2 == startValue)
      for i in range(len(loc[0])):
        startRow = loc[0][i]
        startColumn = loc[1][i]
        selectedArray = np_array2[startRow:(startRow + rowSize1),startColumn:(startColumn + columnSize1)]
        rowSelectedArray = selectedArray.shape[0]
        columnSelectedArray = selectedArray.shape[1]        
        if ((rowSize1 == rowSelectedArray) and (columnSize1 == columnSelectedArray)):
          #print('True')
          check1 = 1
        else:
          #print('halo')
          check1 = 0

  return check1
  #if (check1 == True):
  #  return True
  #else:
  #  return False
  #=========== check table2 in table1
  '''
  if (np_array1.shape[0] > np_array2.shape[0]):
    if (np_array2.shape[1] > np_array1.shape[1]):
      check2 = False
    else:
      startValue = np_array2[0,0]
      loc = np.where(np_array1 == startValue)
      for i in range(len(loc[0])):
        startRow = loc[0][i]
        startColumn = loc[1][i]
        selectedArray = np_array1[startRow:(startRow + rowSize2),startColumn:(startColumn + columnSize2)]
        #print(selectedArray)
        rowSelectedArray = selectedArray.shape[0]
        columnSelectedArray = selectedArray.shape[1]        
        if ((rowSize2 == rowSelectedArray) and (columnSize2 == columnSelectedArray)):
        #print('True')
          check2 = True
        else:
        #print('False')
          check2 = False
  
  if (check1 == False):
    if (check2 == False):
      #print("nggak ada yang sama")
      return 1
    else:
      #print("tabel 2 masuk tabel 1")
      return 2
  else:
    #print("tabel 1 masuk tabel 2")
    return 3
  '''
  

Mehtod removeTitle merupakan method yang berfungsi untuk mendeteks judul yang ada pada tabel tersebut kemudian menghapusnya

In [8]:
def removeTitle(data):
  result = []
  data_array = np.array(data.T.reset_index().values.T.tolist())
  checkTitle, listIndex, data = 0, 0, 0

  counter = 0
  for i in range(len(data_array)):
    #print(counter)
    checkTitle,listIndex = checkNoneData(data_array[i,:], (len(data_array[i,:])))

    #if (i != (len(data_array)-1)):
    if (checkTitle):
      if (i < (len(data_array)-1)):
        if (matchingRow(data_array[i,:], data_array[i+1,:], len(data_array[i,:])-1)):
          if (checkTitle):
            continue
          else:
            result.append(data_array[i,:])
        else:
          if (i < (len(data_array)-2)):
            if (matchingRow3Layer(data_array[i,:], data_array[i+1,:], data_array[i+2,:], len(data_array[i,:])-1)):
              if (checkTitle):
                continue
              else:
                result.append(data_array[i,:])
            else:
              if (i < (len(data_array)-3)):
                if (matchingRow4Layer(data_array[i,:], data_array[i+1,:], data_array[i+2,:], data_array[i+3,:], len(data_array[i,:])-1)):
                  if (checkTitle):
                    continue
                  else:
                    result.append(data_array[i,:])
              else:
                continue
          else:
            continue
      else:
        continue
    else:
      result.append(data_array[i,:])
    counter += 1

  return pd.DataFrame(result[1:], columns = result[0])

def checkNoneData(listData, length):
  counter = 0
  list_index = []
  data = 0

  list_index_nonNull = []
  for i in listData:
    if (str(i) == 'nan' or 'Unnamed' in str(i)):
      list_index.append(counter)
    else:
      list_index_nonNull.append(counter)
    counter += 1

  booleanResult = 0
  
  if ((length-1) in list_index and len(listData) > 3):
    if ((list_index[-1]-list_index[0]+1) == len(list_index) and (len(list_index_nonNull) > 1)):
      booleanResult = False
    elif ((list_index[-1]-list_index[0]+1) == len(list_index)):
      booleanResult = True
    else:
      booleanResult = False
  elif ((0 in list_index) and len(listData) > 3):
    if ((list_index[-1]-list_index[0]+1) == len(list_index) and (len(list_index_nonNull) == 1)):
      booleanResult = True
    else:
      booleanResult = False
  else:
   booleanResult = False
  
  return booleanResult, list_index


Method checkMergeCell merupakan method yang berfungsi untuk menghapus adanya merge cell pada tabel

Pada method ini, merged cell yang bisa diatasi adalah cell yang ter-merge 1-3 baris, untuk cell yang ter-merge lebih dari 3 baris belum bisa di atasi

In [9]:
def checkMergedCell(data):
  data_array = np.array(data.T.reset_index().values.T.tolist())

  #print(matchingRow(data_array[0,:],data_array[1,:]))

  if (len(data_array) > 2):
    if (matchingRow(data_array[0,:],data_array[1,:], len(data_array[0,:])-1)):
    #print(1)
      data_array[0,:] = renameHeader(data_array[0,:], data_array[1,:])
    
      data_result = np.delete(data_array,1,0)
      return pd.DataFrame(data_result[1:], columns = data_result[0])
    else:
      if (len(data_array) > 3):
        if (matchingRow3Layer(data_array[0,:],data_array[1,:], data_array[2,:], len(data_array[0,:])-1)):
    #print(2)
          data_array[1,:] = renameHeader(data_array[1,:], data_array[2,:])
          data_array[0,:] = renameHeader(data_array[0,:], data_array[1,:])
    
          data_result = np.delete(data_array,[1,2],0)
          return pd.DataFrame(data_result[1:], columns = data_result[0])
        else:
          if (len(data_array) > 4):
            if (matchingRow4Layer(data_array[0,:],data_array[1,:], data_array[2,:], data_array[3,:], len(data_array[0,:])-1)):
    #print(2)
              data_array[2,:] = renameHeader(data_array[2,:], data_array[3,:])
              data_array[1,:] = renameHeader(data_array[1,:], data_array[2,:])
              data_array[0,:] = renameHeader(data_array[0,:], data_array[1,:])
    
              data_result = np.delete(data_array,[1,2,3],0)
              return pd.DataFrame(data_result[1:], columns = data_result[0])
            else:
              if (checkColumnMerge(data_array[0,:],data_array[1,:])[0]):
                index = checkColumnMerge(data_array[0,:],data_array[1,:])[1]
                counter = 1
                for i in index:
                  data_array[0,i] = str(data_array[0,i-1]) + "_" + str(counter)
                  counter += 1
    
                data_array[0,:] = renameHeader(data_array[0,:], data_array[1,:])
    
                data_result = np.delete(data_array,1,0)
                return pd.DataFrame(data_result[1:], columns = data_result[0])
              else: 
                return pd.DataFrame(data_array[1:], columns = data_array[0])
          else:
            return pd.DataFrame(data_array[1:], columns = data_array[0])
      else:
        return pd.DataFrame(data_array[1:], columns = data_array[0])
  else:
    #print(3)
    return pd.DataFrame(data_array[1:], columns = data_array[0])

def checkColumnMerge(data1, data2):
  None1, NonNull1 = getNoneIndex(data1)
  None2, NonNull2 = getNoneIndex(data2)

  list_index_noncover = []
  for i in None1:
    if (i in None2):
      list_index_noncover.append(i)
  
  index_merge = []
  for i in None1:
    if (i in NonNull2):
      continue
    else:
      index_merge.append(i)

  list_column_merge = []
  for i in index_merge:
    if (i in list_index_noncover):
      list_column_merge.append(i)

  if (len(list_column_merge) == 0):
    return False,0
  else:
    return True,list_column_merge

def matchingRow(data1, data2, last_index):
  None1, NonNull1 = getNoneIndex(data1)
  None2, NonNull2 = getNoneIndex(data2)
  list_boolean = []

  list_duplicate_index = []
  list_index_nonCover = []

  for i in NonNull1:
    if (i in NonNull2):
      list_duplicate_index.append(i)
  
  if (0 in list_duplicate_index or last_index in list_duplicate_index):
    list_boolean.append(False)
  else:
    for i in list_duplicate_index:
      if (i == 0):
        if ((i+ 1) in list_duplicate_index):
          list_boolean.append(False)
        else:
          list_boolean.append(True)
      elif (i == last_index):
        if ((i-1) in list_duplicate_index):
          list_boolean.append(False)
        else:
          list_boolean.append(True)
      else:
        if ((i-1) in list_duplicate_index or (i+1) in list_duplicate_index):
          list_boolean.append(False)
        else:
          list_boolean.append(True)
  
  cek1 = 0
  for i in None1:
    if (i in NonNull2):
      cek1 = True
    else:
      cek1 = False
  
  if (cek1):
    list_boolean.append(True)
  else:
    list_boolean.append(False)
  
  for i in None2:
    if (i in NonNull1):
      cek1 = True
    else:
      cek1 = False
  
  if (cek1):
    list_boolean.append(True)
  else:
    list_boolean.append(False)

  if (False in list_boolean):
    return False
  else:
    return True  

def matchingRow3Layer(data1, data2, data3, last_index):
  None1, NonNull1 = getNoneIndex(data1)
  None2, NonNull2 = getNoneIndex(data2)
  None3, NonNull3 = getNoneIndex(data3)  

  list_index_duplicate = []
  list_index_nonCover = []
  list_boolean = []

  for i in None2:
    if (i in None3):
      list_index_nonCover.append(i)

  for i in NonNull2:
    if (i in NonNull3):
      list_index_duplicate.append(i)

  list_index_inAllThree = []
  for i in list_index_duplicate:
    if (i in NonNull1):
      list_index_inAllThree.append(i)

#===== cek matching 2 row bawah
  if (0 in list_index_duplicate or last_index in list_index_duplicate):
    list_boolean.append(False)
  else:
    for i in list_index_duplicate:
      if (i == 0):
        if ((i+ 1) in list_index_duplicate):
          list_boolean.append(False)
        else:
          list_boolean.append(True)
      elif (i == last_index):
        if ((i-1) in list_index_duplicate):
          list_boolean.append(False)
        else:
          list_boolean.append(True)
      else:
        if ((i-1) in list_index_duplicate or (i+1) in list_index_duplicate):
          list_boolean.append(False)
        else:
          list_boolean.append(True)

#====== cek matching 3 row bawah
  if (0 in list_index_inAllThree or last_index in list_index_inAllThree):
    list_boolean.append(False)
  else:  
    for i in list_index_inAllThree:
      if (i == 0):
        if (i+1 in list_index_inAllThree):
          list_boolean.append(False)
        else:
          list_boolean.append(True)
      elif (i == last_index):
        if (i-1 in list_index_inAllThree):
          list_boolean.append(False)
        else:
          list_boolean.append(True)
      else:
        if (i-1 in list_index_inAllThree  or i+1 in list_index_inAllThree):
          list_boolean.append(False)
        else:
          list_boolean.append(True)
  
  for i in list_index_nonCover:
    if (i in None1):
      list_boolean.append(False)
    else:
      list_boolean.append(True)
  
  if (False in list_boolean):
    return False
  else:
    return True

def matchingRow4Layer (data1, data2, data3, data4, last_index):
  None1, NonNull1 = getNoneIndex(data1)
  None2, NonNull2 = getNoneIndex(data2)
  None3, NonNull3 = getNoneIndex(data3)
  None4, NonNull4 = getNoneIndex(data4)

  list_index_duplicate = []
  list_index_inAllThree = []
  list_index_inAllFour = []
  list_index_nonCover1 = []
  list_index_nonCover2 = []
  list_boolean = []
  
  for i in None3:
    if (i in None4):
      list_index_nonCover1.append(i)    

  for i in list_index_nonCover1:
    if (i in None2):
      list_index_nonCover2.append(i)
  
  for i in NonNull3:
    if (i in NonNull4):
      list_index_duplicate.append(i)

  for i in list_index_duplicate:
    if (i in NonNull2):
      list_index_inAllThree.append(i)

  for i in list_index_inAllThree:
    if (i in NonNull1):
      list_index_inAllFour.append(i) 

#===== cek matching 2 row bawah
  if (0 in list_index_duplicate or last_index in list_index_duplicate):
    list_boolean.append(False)
  else:
    for i in list_index_duplicate:
      if (i == 0):
        if ((i+ 1) in list_index_duplicate):
          list_boolean.append(False)
        else:
          list_boolean.append(True)
      elif (i == last_index):
        if ((i-1) in list_index_duplicate):
          list_boolean.append(False)
        else:
          list_boolean.append(True)
      else:
        if ((i-1) in list_index_duplicate or (i+1) in list_index_duplicate):
          list_boolean.append(False)
        else:
          list_boolean.append(True)

#====== cek matching 3 row bawah
  if (0 in list_index_inAllThree or last_index in list_index_inAllThree):
    list_boolean.append(False)
  else:  
    for i in list_index_inAllThree:
      if (i == 0):
        if (i+1 in list_index_inAllThree):
          list_boolean.append(False)
        else:
          list_boolean.append(True)
      elif (i == last_index):
        if (i-1 in list_index_inAllThree):
          list_boolean.append(False)
        else:
          list_boolean.append(True)
      else:
        if (i-1 in list_index_inAllThree  or i+1 in list_index_inAllThree):
          list_boolean.append(False)
        else:
          list_boolean.append(True)

#====== cek matching 4 row bawah
  if (0 in list_index_inAllFour or last_index in list_index_inAllFour):
    list_boolean.append(False)
  else:  
    for i in list_index_inAllFour:
      if (i == 0):
        if (i+1 in list_index_inAllFour):
          list_boolean.append(False)
        else:
          list_boolean.append(True)
      elif (i == last_index):
        if (i-1 in list_index_inAllFour):
          list_boolean.append(False)
        else:
          list_boolean.append(True)
      else:
        if (i-1 in list_index_inAllFour or i+1 in list_index_inAllFour):
          list_boolean.append(False)
        else:
          list_boolean.append(True)

  for i in list_index_nonCover2:
    if (i in None1):
      list_boolean.append(False)
    else:
      list_boolean.append(True)

  if (False in list_boolean):
    return False
  else:
    return True  

def getNoneIndex(listData):
  counter = 0 
  listIndexNone = []
  listIndexNonNull = []
  for i in listData:
    if (str(i) == 'nan' or 'Unnamed' in str(i)):
      listIndexNone.append(counter)
    else:
      listIndexNonNull.append(counter)
    counter += 1
  return listIndexNone, listIndexNonNull

def renameHeader(data1, data2):
  indexNull1, indexNonNull1 = getNoneIndex(data1)
  indexNull2, indexNonNull2 = getNoneIndex(data2)

  duplicate_index = []
  data = data1.copy()

  for i in indexNonNull1:
    if (i in indexNonNull2):
      duplicate_index.append(i)

  for i in range(len(data1)):
    if (i in duplicate_index):
      data[i] = str(data2[i]) + "_" + str(data1[i]) 
    elif ((i in indexNull1) and (i in indexNull2)):
      continue
    elif (i in indexNull1):
      index = findIndexMergedCell(duplicate_index, i)
      #print(index)
      data[i] = str(data2[i]) + "_" + str(data1[index])
    else:
      continue
  
  return data


def findIndexMergedCell(duplicateIndex, index):
  for i in range(len(duplicateIndex)):
    if (index == duplicateIndex[i]):
      return index
      break
    else:
      if (i != (len(duplicateIndex)-1)):
        if (index > duplicateIndex[i] and index < duplicateIndex[i+1]):
          return duplicateIndex[i]
        else:
          continue
      else:
        if (index > duplicateIndex[i]):
          return duplicateIndex[i]


In [ ]:
coba = cleaningTable('/content/Produksi Tanaman Sayuran.xlsx')
coba[0][0]

,Provinsi,Bawang Merah (Ton)_Produ,Bawang Putih (Ton)_Produ,Bawang Daun (Ton)_Produk,Kentang (Ton)_Produksi T,Kubis (Ton)_Produksi Tan,Kembang Kol (Ton)_Produk,Petsai/Sawi (Ton)_Produk,Wortel (Ton)_Produksi Ta,Lobak (Ton)_Produksi Tan,Kacang Merah (Ton)_Produ,Kacang Panjang (Ton)_Pro,Cabai Besar (Ton)_Produk,Cabai Rawit (Ton)_Produk,Tomat (Ton)_Produksi Tan,Terung (Ton)_Produksi Ta,Buncis (Ton)_Produksi Ta,Ketimun (Ton)_Produksi T,Labu Siam (Ton)_Produksi,Kangkung (Ton)_Produksi,Bayam (Ton)_Produksi Tan,Melinjo (Ton)_Produksi T,Paprika (Ton)_Produksi T,Jamur (Kg)_Produksi Tana,Petai (Ton)_Produksi Tan,Jengkol (Ton)_Produksi T
0,ACEH,"8 840,00","288,00","994,00","26 529,00","10 457,00","450,00","3 324,00","1 005,00",-,"672,00","10 819,00","63 595,00","61 887,00","20 821,00","9 207,00","1 243,00","15 740,00","5 266,00","7 790,00","3 914,00","10 630,00",-,"26 738,00","453,00","1 209,00"
1,SUMATERA UTARA,"18 072,00","975,00","13 259,00","118 778,00","219 537,00","53 278,00","78 728,00","95 821,00","981,00","1 410,00","29 313,00","154 008,00","49 246,00","118 583,00","60 244,00","28 674,00","22 430,00","21 243,00","13 652,00","16 610,00","2 615,00",-,"37 119,00","8 767,00","4 239,00"
2,SUMATERA BARAT,"122 399,00","1 866,00","43 342,00","50 730,00","131 052,00","15 917,00","35 994,00","41 078,00","619,00","76,00","20 822,00","139 994,00","31 782,00","146 829,00","81 306,00","51 504,00","34 103,00","27 844,00","7 810,00","4 434,00","7 448,00",-,"98 066,00","9 895,00","10 799,00"
3,RIAU,"507,00",-,"3,00",-,-,-,"1 339,00",-,-,-,"9 210,00","17 513,00","8 120,00","117,00","10 225,00","52,00","16 462,00","157,00","9 942,00","8 554,00","1 404,00",-,"49 952,00","1 886,00","3 620,00"
4,JAMBI,"9 686,00","395,00","3 231,00","111 812,00","33 434,00","1 362,00","7 098,00","3 892,00","17,00","1 625,00","7 959,00","42 698,00","9 880,00","12 348,00","10 003,00","4 480,00","5 589,00","1 609,00","5 014,00","3 834,00","3 342,00",-,"16 514,00","1 603,00","4 177,00"
5,SUMATERA SELATAN,"1 390,00","157,00","2 296,00","672,00","6 847,00","124,00","4 141,00","3 669,00",-,"74,00","9 755,00","40 479,00","11 014,00","12 487,00","16 538,00","6 955,00","10 877,00","5 804,00","6 729,00","3 627,00","1 481,00",-,"205 732,00","3 788,00","6 557,00"
6,BENGKULU,"523,00","137,00","16 884,00","4 093,00","66 021,00","10 761,00","29 270,00","35 444,00",-,"1 047,00","4 919,00","37 812,00","11 118,00","18 485,00","53 267,00","24 033,00","9 988,00","8 133,00","3 009,00","670,00","627,00",-,"14 003,00","1 907,00","4 432,00"
7,LAMPUNG,"3 634,00","49,00","3 227,00","297,00","7 725,00","299,00","9 095,00","6 155,00",-,"514,00","12 465,00","40 101,00","12 796,00","18 669,00","18 312,00","6 140,00","10 330,00","5 153,00","12 211,00","7 489,00","14 108,00","1,00","175 623,00","18 003,00","15 363,00"
8,KEP. BANGKA BELITUNG,"170,00",-,"172,00",-,-,-,"1 521,00",-,"193,00",-,"1 817,00","2 468,00","2 002,00","473,00","2 872,00","367,00","3 688,00","271,00","2 024,00","1 225,00","300,00",-,"2 119,00","734,00","152,00"
9,KEP. RIAU,"95,00",-,"396,00",-,-,-,"3 843,00",-,-,-,"4 550,00","4 351,00","2 339,00","285,00","2 575,00","692,00","5 559,00",-,"5 214,00","4 198,00","258,00",-,"2 052,00","648,00","547,00"


In [40]:
#tes = np.array(coba[0][0].T.reset_index().values.T.tolist())

#cobacoba = getIndexColumn(tes[1,:])
#print(cobacoba)
#for i in range(tes.shape[0]):
#  for y in range(tes.shape[1]):
#    if (tes[i,y] == None or tes[i,y] == 'NaN' or tes[i,y] == 'nan'):
#      continue
#    else:
#      print('halo')
#matchingRow3Layer(tes[0,:], tes[1,:], tes[2,:])
#tes = similarityTable(coba[0][0], coba[0][1])
#print(tes)
#print(removeNoneTable(coba[0][0]))
coba = cleaningTablebySheetname('/content/Jumlah_kerusakan_hutan_berdasar_banyaknya_kerugian.xlsx', sheet_Name ='Sheet1')
coba[0]

,Kabupaten/Kota,2016_Pencurian Kayu_Jumlah kerusakan hutan berdasa,2015_Pencurian Kayu_Jumlah kerusakan hutan berdasa,2014_Pencurian Kayu_Jumlah kerusakan hutan berdasa,2016_Bencana Alam_Jumlah kerusakan hutan berdasar,2015_Bencana Alam_Jumlah kerusakan hutan berdasar,2014_Bencana Alam_Jumlah kerusakan hutan berdasar,2016_Kebakaran_Jumlah kerusakan hutan berdasar ban,2015_Kebakaran_Jumlah kerusakan hutan berdasar ban,2014_Kebakaran_Jumlah kerusakan hutan berdasar ban
0,D.I. Yogyakarta,"47,30","39,15","108,37","7,30","16,92","18,21",-,"27,95","26,08"
1,Kulonprogo,-,-,-,"1,56","1,11",-,-,-,-
2,Bantul,"0,52","2,53","3,07","1,59",-,"1,39",-,-,"0,40"
3,Gunungkidul,"46,78","36,62","105,30","4,15","14,91","16,82",-,"1,50","25,68"
4,Sleman,-,-,-,-,-,-,-,-,-
5,Yogyakarta,-,-,-,-,-,-,-,-,-
